In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, TensorDataset, DataLoader


### VIT Reference
Refer to (https://github.com/Ishushan02/Cervical-Cancer-Detection-Using-Vision-Transformer) for Vision Transformer block code

In [5]:
x = torch.arange(0, 32).view(2, 1, 4, 4) 
print(x)
patches = x.unfold(2, 2, 2).unfold(3, 2, 2)
print(patches)


tensor([[[[ 0,  1,  2,  3],
          [ 4,  5,  6,  7],
          [ 8,  9, 10, 11],
          [12, 13, 14, 15]]],


        [[[16, 17, 18, 19],
          [20, 21, 22, 23],
          [24, 25, 26, 27],
          [28, 29, 30, 31]]]])
tensor([[[[[[ 0,  1],
            [ 4,  5]],

           [[ 2,  3],
            [ 6,  7]]],


          [[[ 8,  9],
            [12, 13]],

           [[10, 11],
            [14, 15]]]]],




        [[[[[16, 17],
            [20, 21]],

           [[18, 19],
            [22, 23]]],


          [[[24, 25],
            [28, 29]],

           [[26, 27],
            [30, 31]]]]]])


In [16]:
class MakeVideoDataset(Dataset):

    def __init__(self, video, input_length = 10, output_length = 10):
        super().__init__()
        self.video = video
        self.input_length = input_length
        self.output_length = output_length
    
    def __len__(self):
        return self.video.shape[0] - self.input_length - self.output_length + 1
    
    def __getitem__(self, idx):
        input_frame = self.video[idx:idx+self.input_length]
        output_frame = self.video[idx+self.input_length: idx+self.input_length+self.output_length]
        return  (
            torch.tensor(input_frame, dtype=torch.float32),
            torch.tensor(output_frame, dtype=torch.float32)
        )
    
    
test = torch.randn(20, 1, 4, 4)
out = MakeVideoDataset(test)
x, y = out.__getitem__(0)
x.shape, y.shape

/var/folders/y8/q2s37ndx6tg3lpzp3vp8xk_r0000gn/T/ipykernel_27909/4074788471.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(input_frame, dtype=torch.float32),
/var/folders/y8/q2s37ndx6tg3lpzp3vp8xk_r0000gn/T/ipykernel_27909/4074788471.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(output_frame, dtype=torch.float32)


(torch.Size([10, 1, 4, 4]), torch.Size([10, 1, 4, 4]))